In [1]:
import os
import tempfile
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

plt.style.use('seaborn-whitegrid')

2023-04-26 13:35:52.734762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_2078663/1293145838.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
import os
import glob
from pymongo import MongoClient
import pandas as pd

database = "vnw_job"
col = "items"

mongo = MongoClient("mongodb://10.122.6.17:27017")
df_meta = pd.DataFrame(list(mongo[database][col].find()))
df_meta['jobTitle'] = df_meta.categoricalProps.apply(lambda x: x["jobTitle"][0])
df_meta['skills'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["skills"]))
df_meta['industries'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["industries"]))
df_meta['locations'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["locations"]))
df_meta['jobLevel'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["jobLevel"]))
df_meta['availableDate'] = df_meta.dateProps.apply(lambda x: x["availableDate"])
df = pd.read_parquet(glob.glob("/home/spark/ylv/data/navidata/*.parquet"))

In [3]:
ratings = df.merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")

In [4]:
ratings = ratings.fillna('UNK')

In [5]:
%%time

movies = tf.data.Dataset.from_tensor_slices({
    "job_id": df_meta._id.values.tolist(),
    "category": df_meta.industries.values.tolist(),
    "location": df_meta.locations.values.tolist(),
    "level": df_meta.jobLevel.values.tolist(),
    "job_title": df_meta.jobTitle.values.tolist(),
    "skill_text": df_meta.skills.values.tolist()
})

tensor_slices = {
    "user_id": ratings.entityId.values.tolist(),
    "job_id": ratings.targetEntityId.values.tolist(),
    "location": ratings.locations.values.tolist(),
    "category": ratings.industries.values.tolist(),
    "level": ratings.jobLevel.values.tolist(),
    "job_title": ratings.jobTitle.values.tolist(),
    "skill_text": ratings.skills.values.tolist()
}

2023-04-26 13:36:27.478355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 13:36:27.506353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 13:36:27.506505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 13:36:27.506944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

CPU times: user 814 ms, sys: 365 ms, total: 1.18 s
Wall time: 1.19 s


In [6]:
%%time

ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

CPU times: user 1min 4s, sys: 2.05 s, total: 1min 6s
Wall time: 1min 6s


In [7]:
%%time

from functools import reduce

unique_job_titles = np.unique(df_meta.jobTitle.values.tolist())
unique_job_ids = np.unique(df_meta._id.values.tolist())
unique_user_ids = np.unique(df.entityId.values.tolist())
unique_locations = np.unique(reduce(lambda x, y: x + "," + y, df_meta.locations.values.tolist()).split(","))
unique_skills = np.unique(reduce(lambda x, y: x + "," + y, df_meta.skills.values.tolist()).split(","))
# job_title_vocabs = np.unique(reduce(lambda x, y: x + y, df_meta.title.str.split(" ").tolist()))
unique_category = np.unique(reduce(lambda x, y: x + "," + y, df_meta.industries.values.tolist()).split(","))

CPU times: user 7.21 s, sys: 1.49 s, total: 8.7 s
Wall time: 8.69 s


In [8]:
%%time

unique_levels = np.unique(reduce(lambda x, y: x + "," + y, df_meta.jobLevel.values.tolist()).split(","))

CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 123 ms


In [9]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"])
    ], axis=1)

In [10]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    print(feature_embedding)
    return self.dense_layers(feature_embedding)

In [11]:
def tokenization(t):
    return tf.strings.split(t, ',')

def tokenization_1(t):
    return tf.strings.split(t, ' ')


class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.job_id_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_job_ids,mask_token=None),
      tf.keras.layers.Embedding(len(unique_job_ids) + 1, 32)
    ])
    self.location_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_locations,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_locations) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    self.level_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_levels,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_levels) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    self.category_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_category,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_category) + 1, 64),
      tf.keras.layers.GlobalAvgPool1D()
    ])
#     self.skill_embedding = tf.keras.Sequential([
#       tf.keras.layers.TextVectorization(
#           max_tokens=100,
#           standardize=None,
#           split=tokenization,
#           vocabulary=unique_skills,
#           pad_to_max_tokens=True
#       ),
#       tf.keras.layers.Embedding(len(unique_skills) + 1, 32),
#       tf.keras.layers.GlobalAvgPool1D()
#     ])

  def call(self, features):
    return tf.concat([
        self.job_id_embedding(features["job_id"]),
        self.category_embedding(features["category"]),
        self.location_embedding(features["location"]),
        self.level_embedding(features["level"]),
#         self.skill_embedding(features["skill_text"])
    ], axis=1)

In [12]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [13]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"]
    })
    movie_embeddings = self.candidate_model({
        "job_id": features["job_id"],
        "job_title": features["job_title"],
        "category": features["category"],
        "location": features["location"],
        "skill_text": features["skill_text"],
        "level": features["level"],
    })

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [14]:
cached_train = ratings.batch(4096)

In [15]:
%%time

num_epochs = 50

model = MovielensModel([128, 64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

Tensor("query_model/user_model/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("query_model/user_model/concat/concat:0", shape=(None, 32), dtype=float32)


2023-04-26 13:38:11.403762: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x99727170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-26 13:38:11.403780: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-04-26 13:38:11.406672: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-26 13:38:11.447452: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:11.472086: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster u

2023-04-26 13:38:13.334253: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:13.404983: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:13.476509: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-04-26 13:38:15.161054: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:15.239721: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:15.310495: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-04-26 13:38:17.002645: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:17.079907: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:38:17.151722: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

...

2023-04-26 13:40:57.655378: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:40:57.742984: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:40:57.823780: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-04-26 13:40:59.652649: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:40:59.731690: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:40:59.811955: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-04-26 13:41:01.612352: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:41:01.717594: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:41:01.838889: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-04-26 13:41:03.853118: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:41:03.964412: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-26 13:41:04.060985: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

CPU times: user 45min 30s, sys: 1min 25s, total: 46min 55s
Wall time: 20min 30s


In [19]:
one_layer_history = model.fit(
    cached_train,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

In [16]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=10)
index.index_from_dataset(
    movies.batch(500).map(lambda x: (x["job_id"], model.candidate_model(x))))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [17]:
score, pred = index({"user_id": np.array(["3788012"])})
list(map(lambda x: x.decode('ascii'), pred.numpy()[0]))
df_meta[df_meta._id.isin(list(map(lambda x: x.decode('ascii'), pred.numpy()[0])))][["jobTitle", "industries", "locations", "jobLevel", "skills"]]

tf.Tensor(
[[-0.06353951  0.16869582 -0.6216657   0.18809833 -0.0739537   0.22784534
   0.30348593  0.31435144 -0.1808117   0.49270895 -0.19162887 -0.09025189
  -0.5027668  -0.03434692  0.15173687  0.16671248 -0.18251655 -0.20988336
   0.11893692  0.55175346  0.1001528  -0.15083367 -0.2203451  -0.2554129
  -0.04558424 -0.01655666  0.3454199   0.04281245  0.6488379   0.13659628
  -0.28268808  0.12365586]], shape=(1, 32), dtype=float32)


,jobTitle,industries,locations,jobLevel,skills
7445,finance manager,"Accounting,Auditing,Finance/Investment","Dong Nai,Ho Chi Minh",Manager,"financial report analysis,finance management,f..."
7864,assistant manager finance account,"Consulting,Retail/Wholesale",Ho Chi Minh,Experienced (non-manager),"finance,accounting standards,sap"
9111,manager finance ifrs17,"Accounting,Insurance,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"ifrs standard,analytical skill,project managem..."
9247,finance manager,"Accounting,Finance/Investment",Ho Chi Minh,Manager,"finance,finance analysis,tax accounting"
11050,finance manager,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"finance management,retail industry,cashflow ma..."
11329,commercial finance manager,"Accounting,Finance/Investment,Fintech",Ho Chi Minh,Experienced (non-manager),"finance,finance business partnering,deployment..."
11467,assistant manager finance management,"Banking,Auditing,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"finance,audit,finance analysis,business admini..."
11863,corporate finance manager,"Accounting,Executive management,Finance/Invest...",Ho Chi Minh,Experienced (non-manager),"finance,cfa,cpa,corporate finance,finance data..."
12047,lead financial accountant manager level,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"computer skill,mathematics analysis,financial ..."
12160,financial controller,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"finance controling,cpa,accounting,finance,kiem..."


In [46]:
import random
u = random.choice(unique_user_ids)
# u = "14.186.104.233, 10.122.10.10"
print("=====================HISTORY=======================")
df[df.entityId==u].merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")[["jobTitle", "industries", "locations", "jobLevel", "_id"]]

=====================HISTORY=======================


,jobTitle,industries,locations,jobLevel,_id
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,sales distribution analyst,"Marketing,Sales,Auto/Automotive",Ha Noi,Experienced (non-manager),1622149
4,giam doc kinh doanh,"Executive management,Sales,Sales Technical",Ha Noi,Director and above,1625086
5,key account sales executive,"Retail/Wholesale,Sales,HVAC",Ho Chi Minh,Experienced (non-manager),1626079
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,chuyen vien kinh doanh tai vietnamworks,"Customer Service,Education/Training,Sales","Ha Noi,Ho Chi Minh",Experienced (non-manager),1622373


In [47]:
score, pred = index({"user_id": np.array([u])})
list(map(lambda x: x.decode('ascii'), pred.numpy()[0]))

print("=====================PREDICT=======================")
a_1 = list(map(lambda x: x.decode('ascii'), pred.numpy()[0]))
a = df_meta[df_meta._id.isin(list(map(lambda x: x.decode('ascii'), pred.numpy()[0])))][["jobTitle", "industries", "locations", "jobLevel", "skills", "_id"]]
# a["s_value"] = a.
a["s_value"] = a._id.apply(lambda x: a_1.index(x))
a.sort_values("s_value")

tf.Tensor(
[[-0.12686442  0.0506655   0.1226304  -0.4312762   0.04562857  0.38137853
  -0.08054727  0.1946694  -0.05013992  0.01500891 -0.10834578 -0.00189344
   0.11389492  0.09633329 -0.10372746 -0.15265201  0.10796867  0.35015824
  -0.03046202 -0.17793877 -0.27715778  0.13888195 -0.12518868  0.05468545
   0.00763723 -0.4026675  -0.02950197 -0.34562972 -0.01447106 -0.07868527
   0.08335356 -0.1570562 ]], shape=(1, 32), dtype=float32)
=====================PREDICT=======================


,jobTitle,industries,locations,jobLevel,skills,_id,s_value
11767,leader creative tiktok,"Advertising/Promotion/PR,Marketing,Internet/On...",Ha Noi,Manager,"creative ideas,r d formulation,phat trien san ...",1635056,0
11064,sales leader dich vu event tech services,"Sales Technical,TV/Media/Newspaper,IT - Hardwa...",Ha Noi,Experienced (non-manager),"kinh doanh,ban hang ky thuat,events sales",1634145,1
12093,sales engineer industrial,"Sales Technical,Mechanical,Industrial Products",Ha Noi,Experienced (non-manager),"mechanical engineering,industrial system,sales...",1635437,2
6431,trade marketing manager,"Advertising/Promotion/PR,Executive management,...",Ha Noi,Manager,"trade marketing,quan ly du an,lap ke hoach mar...",1628381,3
11684,truong phong kinh doanh spcn,"Marketing,Sales,Sales Technical",Ha Noi,Manager,"tieng anh,tim kiem khach hang,tu van san pham,...",1634658,4
3375,business development executive sale executive ...,"Customer Service,Marketing,Sales",Ha Noi,Experienced (non-manager),"business development,english,marketing develop...",1624528,5
8462,design consult manager hvac background,"Sales Technical,Mechanical,HVAC",Ho Chi Minh,Manager,"hvac design,hvac system,consulting clients,sal...",1630867,6
12143,nhan vien kinh doanh noi that,"Architecture/Interior Design,Sales,Sales Techn...",Ha Noi,Experienced (non-manager),"kinh doanh noi that,dam phan,khai thac thong t...",1635391,7
10396,kinh doanh thiet bi cong nghiep,"Marketing,Sales,Sales Technical",Ha Noi,Experienced (non-manager),"phat trien thi truong,kinh doanh,thiet bi cong...",1633232,8
6752,chuyen vien quan ly ban hang khu vuc,"Sales,Sales Technical,Auto/Automotive",Ha Noi,Experienced (non-manager),"area sales,area management,automotive industry...",1628765,9


In [307]:
u

'14.224.158.205, 10.122.10.10'

In [24]:
a = model.candidate_model.embedding_model.category_embedding.get_layer("text_vectorization_2")(np.array(["Pharmaceutical/Biotech,Retail/Wholesale,FMCG"]))
print(a)
model.candidate_model.embedding_model.category_embedding.get_layer("embedding_4")(a)[0][0]

tf.Tensor([[57 65 26]], shape=(1, 3), dtype=int64)


<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 8.20102915e-02, -1.09246433e-01,  1.16406173e-01, -2.15927066e-04,
       -2.17912402e-02,  2.16943428e-01, -4.32167590e-01, -1.50346830e-01,
       -1.55720711e-01,  1.90026164e-02, -8.97300616e-02,  2.36291751e-01,
       -1.15746878e-01,  1.18816935e-01, -1.39178023e-01,  1.63792342e-01,
       -4.02456999e-01, -5.30271828e-01,  3.26807827e-01, -8.69098455e-02,
        2.82072872e-01,  2.89442748e-01, -5.01493812e-01,  8.40007290e-02,
       -3.64728242e-01, -2.43987009e-01,  7.98815936e-02, -6.21027760e-02,
       -4.10830259e-01, -3.61638606e-01,  1.97913647e-01,  5.56449234e-01,
       -7.83265010e-02,  1.88678131e-03, -1.77316412e-01,  6.67164475e-02,
       -2.96052366e-01,  7.22513497e-02, -1.77137554e-01, -3.57891500e-01,
       -2.05979660e-01, -1.15825750e-01, -2.76240170e-01,  3.85873377e-01,
       -3.29503492e-02, -1.43369347e-01, -9.62817017e-03,  2.59953707e-01,
        2.66019721e-02, -9.55201238e-02, -1.15406504e

In [25]:
mv = model.candidate_model.embedding_model.category_embedding.get_layer("text_vectorization_2").get_vocabulary()
mv.index("Marketing")

49

In [27]:
model.candidate_model.embedding_model.category_embedding.get_layer("embedding_4").get_weights()[0][49] == model.candidate_model.embedding_model.category_embedding.get_layer("embedding_4")(a)[0][0]

<tf.Tensor: shape=(64,), dtype=bool, numpy=
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])>

In [102]:
embedding_cat = model.candidate_model.embedding_model.category_embedding.get_layer("embedding_19").get_weights()[0]
embedding_cat[7]

array([ 0.04258758, -0.07968332, -0.05580219, -0.02753802, -0.0101841 ,
       -0.12104866, -0.3190667 , -0.04764057,  0.13371506,  0.01419604,
        0.3656076 ,  0.19303547, -0.01793406, -0.09390188,  0.07415319,
       -0.36882246, -0.08373842,  0.6550277 ,  0.16837431, -0.2304274 ,
       -0.06157956,  0.07596666, -0.08147656, -0.39094543, -0.04607748,
        0.08323601, -0.11870127, -0.2871983 ,  0.03669569,  0.3004049 ,
       -0.10802484, -0.16634753, -0.03408201,  0.26850766,  0.19464758,
       -0.16935734, -0.0497155 ,  0.00220176,  0.15710092,  0.02497763,
       -0.04698634, -0.0928045 ,  0.04975066, -0.05204397, -0.10860399,
       -0.10967059, -0.62644887,  0.02970633, -0.27730337, -0.41667613,
       -0.08826417,  0.17892775,  0.14095348,  0.12049776,  0.20826578,
       -0.01085067, -0.00512729, -0.13454823, -0.14827728,  0.44416896,
       -0.24629797,  0.10795631, -0.01696168, -0.24137892], dtype=float32)

In [157]:
# b = model.candidate_model.embedding_model.category_embedding


<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[ 2.8321410e-02, -2.5932738e-04,  1.4152543e-01,  7.6472752e-02,
         4.9156379e-02, -1.7751646e-01,  4.4903379e-02, -2.7023196e-02,
        -6.5955840e-02,  9.5899170e-03, -3.5046164e-03,  8.1099570e-02,
        -1.2325066e-01, -1.0400704e-01,  8.9547314e-02, -1.2544177e-01,
        -5.1424563e-02,  1.4805318e-01,  1.2710010e-02, -2.4851492e-02,
         2.7700929e-02,  4.3086997e-01, -6.5218009e-02,  1.5512338e-01,
         5.5250246e-02, -2.2489185e-02, -1.2831549e-02,  1.6805655e-01,
         6.2538169e-02, -5.7037991e-02, -3.3218920e-01,  7.8225173e-02,
        -6.4002089e-02,  1.7894553e-01,  1.4037323e-01, -6.7494869e-02,
         2.2491981e-01, -7.9892375e-02,  2.6967895e-01,  8.3900839e-02,
         2.4375486e-01,  1.6539170e-01,  4.9552095e-01, -2.2304043e-02,
        -4.4861332e-01,  6.5109365e-02,  1.9142826e-01, -1.0420974e-01,
        -9.8404191e-02, -4.2973816e-01, -1.3990875e-01, -1.4972560e-01,
         1.3697

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

industries = df_meta.industries.unique()
n = random.randint(0, len(industries) + 1)
print(len(industries))
print(industries[n])
w = model.candidate_model.embedding_model.category_embedding(industries)
[industries[i] for i in list(reversed(np.argsort(cosine_similarity(np.array([w[n]]), w))[0]))][:20]

3604
Customer Service,Marketing,Sales


['Customer Service,Marketing,Sales',
 'Advertising/Promotion/PR,Customer Service,Sales',
 'Customer Service,Sales,Luxury Goods',
 'Customer Service,Sales,E-commerce',
 'Retail/Wholesale,Sales,Internet/Online Media',
 'Customer Service,Retail/Wholesale,Sales Technical',
 'Customer Service,Sales,Fashion/Lifestyle',
 'Consulting,Marketing,Sales',
 'Advertising/Promotion/PR,Customer Service,Sales Technical',
 'Marketing,Sales,Household',
 'Customer Service,Marketing,Sales Technical',
 'Customer Service,Sales,Printing',
 'Advertising/Promotion/PR,Customer Service,Retail/Wholesale',
 'Customer Service,Retail/Wholesale,Sales',
 'Marketing,Retail/Wholesale,Sales',
 'Insurance,Marketing,Sales',
 'Marketing,Sales,Airlines/Tourism',
 'Retail/Wholesale,Sales,Printing',
 'Customer Service,Sales',
 'Marketing,Sales,Interpreter/Translator']

In [182]:
df_meta.head(1)[["jobTitle", "industries", "locations", "jobLevel", "skills", "_id"]].values

array([['content manager truong bo phan xay dung chuong trinh va hoc lieu',
        'Education/Training,Executive management,Marketing', 'Ha Noi',
        'Manager',
        'pedagogy skill,communnication skill,english,organizational management,teaching support',
        '1603801']], dtype=object)

In [41]:
i_embedding = model.candidate_model.embedding_model({
        "job_id": df_meta._id.values,
        "job_title": df_meta.jobTitle.values,
        "category": df_meta.industries.values,
        "location": df_meta.locations.values,
        "skill_text": df_meta.skills.values,
        "level": df_meta.jobLevel.values
})

list(reversed(np.argsort(cosine_similarity(np.array([i_embedding[0]]), i_embedding)[0])))[:10]

[0, 18310, 18399, 18308, 17782, 15820, 13536, 15578, 15072, 15084]

In [45]:
n = random.randint(0, df_meta.shape[0])
e1 = list(reversed(np.argsort(cosine_similarity(np.array([i_embedding[n]]), i_embedding)[0])))[:20]
e = df_meta[df_meta.index.isin(e1)][["jobTitle", "industries", "locations", "jobLevel", "skills", "_id"]]
e["s_value"] = e.index
e["s_value"] = e.s_value.apply(lambda x: e1.index(x))
e.sort_values("s_value")

,jobTitle,industries,locations,jobLevel,skills,_id,s_value
17021,truong phong kinh doanh,"Executive management,Sales,Food & Beverage",Tien Giang,Manager,"dam phan kinh doanh,quan ly kinh doanh,kinh do...",1641572,0
17133,sale manager,"Executive management,Sales,Food & Beverage",Ho Chi Minh,Manager,"english,analytical skill,relationship manageme...",1641644,1
16205,sales manager mekong,"Sales,FMCG",Can Tho,Manager,"b2b sales,fmcg",1640619,2
18488,giam doc ban hang chi ung vien thuong tru bac ...,"Executive management,Retail/Wholesale,Sales",Bac Giang,Manager,"quang ba tiep thi,chien luoc kinh doanh,phan t...",1643372,3
18960,sales manager,"Executive management,Retail/Wholesale,Sales",Ho Chi Minh,Manager,"sale b2b,b2c sales,cosmetic,retail,sale leader",1644049,4
17428,sales manager css south north vn,"Executive management,Retail/Wholesale,Sales",Ho Chi Minh,Manager,"sales planning,sales strategy,project manageme...",1642117,5
15954,sale manager,"Executive management,Retail/Wholesale,Sales",Binh Duong,Manager,"phat trien chuoi,kenh mt,fmcg,asm fmcg,leadership",1640229,6
17943,business unit manager operations manager,"Executive management,Sales,Food & Beverage",Ha Noi,Manager,"business unit management,leadership,business d...",1642746,7
14724,giam sat ban hang binh phuoc,"Retail/Wholesale,Sales,FMCG",Binh Phuoc,Manager,"dieu hanh kinh doanh,giam sat ban hang,fmcg,sa...",1638761,8
17964,giam sat ban hang gt ninh kieu can tho,"Retail/Wholesale,Sales,FMCG",Binh Phuoc,Manager,"giam sat ban hang,kenh gt,giam sat ban le,fmcg",1642788,9


In [250]:
# model.candidate_model.embedding_model.location_embedding(np.array)
# t_1 = model.candidate_model.embedding_model.location_embedding(df_meta.locations.values[:1])
# t = model.candidate_model.embedding_model.location_embedding(df_meta.locations.values)

print(model.candidate_model.embedding_model.location_embedding.get_layer("text_vectorization_9")(np.array(["Ha Noi", "Ho Chi Minh"])))
model.candidate_model.embedding_model.location_embedding.get_layer("text_vectorization_9")(np.array([""]))

tf.Tensor(
[[23]
 [28]], shape=(2, 1), dtype=int64)


<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[0]])>

In [223]:
unique_locations

array(['An Giang', 'Ba Ria - Vung Tau', 'Bac Giang', 'Bac Lieu',
       'Bac Ninh', 'Ben Tre', 'Binh Dinh', 'Binh Duong', 'Binh Phuoc',
       'Binh Thuan', 'Ca Mau', 'Can Tho', 'Da Nang', 'Dak Lak',
       'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai',
       'Ha Giang', 'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong',
       'Hai Phong', 'Hau Giang', 'Ho Chi Minh', 'Hoa Binh', 'Hung Yen',
       'International', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
       'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Mekong Delta',
       'Nam Dinh', 'Nghe An', 'Ninh Binh', 'Ninh Thuan', 'Other',
       'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam', 'Quang Ngai',
       'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
       'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien Hue',
       'Tien Giang', 'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc',
       'Yen Bai'], dtype='<U17')

In [319]:
path = "/home/spark/ylv/recommend_search_project/navi_model"
tf.saved_model.save(
    index,
    path,
    options=tf.saved_model.SaveOptions(namespace_whitelist=["brute_force_2"])
)

Tensor("brute_force_3/query_model_3/user_model_3/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("query_model_3/user_model_3/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("query_model_3/user_model_3/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_3/concat/concat:0", shape=(None, 32), dtype=float32)


INFO:tensorflow:Assets written to: /home/spark/ylv/recommend_search_project/navi_model/assets


INFO:tensorflow:Assets written to: /home/spark/ylv/recommend_search_project/navi_model/assets


In [320]:
loaded_model = tf.saved_model.load(path)

In [322]:
index = tfrs.layers.factorized_top_k.BruteForce(loaded_model.query_model, k=10)
index.index_from_dataset(
    movies.batch(500).map(lambda x: (x["job_id"], loaded_model.candidate_model(x))))

AttributeError: in user code:

    File "/tmp/ipykernel_1950944/517015533.py", line 3, in None  *
        lambda x: (x["job_id"], loaded_model.candidate_model(x))

    AttributeError: '_UserObject' object has no attribute 'candidate_model'
